In [60]:
import torch
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from tokenizers import Tokenizer
from torch import nn
import torch.nn.functional as F
import sys
from transformers import pipeline 

TOKENIZER = "C:\Emotion Classification\\traning\\vocab\\tokenizer0.2.json"

tokenizer = Tokenizer.from_file(TOKENIZER)

<>:10: SyntaxWarning: invalid escape sequence '\E'
<>:10: SyntaxWarning: invalid escape sequence '\E'
C:\Users\illya\AppData\Local\Temp\ipykernel_20784\422757640.py:10: SyntaxWarning: invalid escape sequence '\E'
  TOKENIZER = "C:\Emotion Classification\\traning\\vocab\\tokenizer0.2.json"
c:\Emotion Classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, num_classes, dropout=0.3):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        self.lstm = nn.LSTM(
            embedding_dim, 
            hidden_size, 
            num_layers=num_layers,
            batch_first=True, 
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=False
        )
        self.dropout1 = nn.Dropout(dropout)
        
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.bn1 = nn.BatchNorm1d(hidden_size // 2)
        self.dropout2 = nn.Dropout(dropout)
        
        self.fc2 = nn.Linear(hidden_size // 2, hidden_size // 4)
        self.bn2 = nn.BatchNorm1d(hidden_size // 4)
        self.dropout3 = nn.Dropout(dropout * 0.5)
        

        self.output_layer = nn.Linear(hidden_size // 4, num_classes)
        
    def forward(self, x):

        embedded = self.embedding(x)
        

        lstm_out, _ = self.lstm(embedded)
        

        out = lstm_out.mean(dim=1)
        

        out = self.dropout1(out)

        out = self.fc1(out)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.dropout2(out)
        
        out = self.fc2(out)
        out = self.bn2(out)
        out = F.relu(out)
        out = self.dropout3(out)

        out = self.output_layer(out)
        
        return out

In [53]:
sequence_len = 62
input_len = 62
hidden_size = 256
num_layers = 1
num_classes = 13
num_epochs = 10
learning_rate = 0.01

In [54]:
vocab_size = tokenizer.get_vocab_size()

In [108]:
generator = pipeline("text-classification")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [226]:
text_to_text = "hope to see you again"
text = {'text': text_to_text}
text_to_text = pd.DataFrame(data=text, index=[0])
text_to_text

,text
0,hope to see you again


In [227]:
first_branch = generator(text_to_text.text[0])
first_branch = list(first_branch[0].values())[0]


if first_branch == 'POSITIVE':
    model = torch.load('C:\Emotion Classification\\traning\\train0.3\models\Pmodel.pth', weights_only=False)
    em = ['text', 'admiration', 'amusement', 'approval', 'caring', 'curiosity',
       'desire', 'excitement', 'gratitude', 'joy', 'love', 'optimism', 'pride',
       'relief']
    print("POS")
else:
    model = torch.load(r'C:\Emotion Classification\\traning\\train0.3\models\Nmodel.pth', weights_only=False)
    em = ['anger', 'annoyance', 'confusion', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'fear', 'grief',
       'nervousness', 'remorse', 'sadness']
    print("NEG")

POS


<>:6: SyntaxWarning: invalid escape sequence '\E'
<>:6: SyntaxWarning: invalid escape sequence '\E'
C:\Users\illya\AppData\Local\Temp\ipykernel_20784\1025466880.py:6: SyntaxWarning: invalid escape sequence '\E'
  model = torch.load('C:\Emotion Classification\\traning\\train0.3\models\Pmodel.pth', weights_only=False)


In [228]:
model = LSTMModel(vocab_size, hidden_size, hidden_size, num_layers, num_classes)

In [229]:
def prepare_text_for_inference(df=text_to_text, tokenizer=tokenizer):

    df = df.copy()
    tokenized = df['text'].apply(lambda x: tokenizer.encode(x).ids)
    tensor_list = [torch.tensor(seq, dtype=torch.long) for seq in tokenized.tolist()]
    X = pad_sequence(tensor_list, batch_first=True, padding_value=0)
    
    return X

X_new = prepare_text_for_inference()

In [230]:
model.eval()
with torch.no_grad():
    outputs = model(X_new)
    probs = torch.sigmoid(outputs)
    preds = (probs > 0.5).int()
    
preds_list = preds.tolist()

for i in range(len(preds_list[0])):
    if preds_list[0][i] == 1:
        result = preds_list[0].index(preds_list[0][i]) + 1
        print("working")

em[result]

working
working
working
working
working
working
working
working


'admiration'